In [1]:
import psycopg2
import pandas as pd
import uuid

In [2]:
dbhost = "localhost"
dbname = "db_02"
dbuser = "postgres"
dbpassword = "ABcd@12#$"

source = 'creditcard_test'

In [74]:
file_data = pd.read_csv(f"../base files/{source}.csv")
# print(len(file_data))

try:
    conn = psycopg2.connect(dbname=dbname, user=dbuser, password=dbpassword, host=dbhost)
except psycopg2.Error as e:
    print("Error connecting to database:", e)
    exit()

if(conn):
    print('connected')

cur = conn.cursor()

connected


In [75]:
Arr=[]
for i in range (1, 29):
    Arr.append(f"V{i}")

columns = ", ".join(Arr)
creation_columns=" DECIMAL, ".join(Arr)

print(creation_columns)

V1 DECIMAL, V2 DECIMAL, V3 DECIMAL, V4 DECIMAL, V5 DECIMAL, V6 DECIMAL, V7 DECIMAL, V8 DECIMAL, V9 DECIMAL, V10 DECIMAL, V11 DECIMAL, V12 DECIMAL, V13 DECIMAL, V14 DECIMAL, V15 DECIMAL, V16 DECIMAL, V17 DECIMAL, V18 DECIMAL, V19 DECIMAL, V20 DECIMAL, V21 DECIMAL, V22 DECIMAL, V23 DECIMAL, V24 DECIMAL, V25 DECIMAL, V26 DECIMAL, V27 DECIMAL, V28


In [ ]:
# cur.execute(f"DROP TABLE {source};")
# conn.commit()
# cur.close()
# conn.close()

In [ ]:
cur.execute(f"CREATE TABLE {source} (ID CHAR(36), Time INTEGER, {creation_columns} DECIMAL, Amount INTEGER, Class INTEGER);")

cur.execute(f"SELECT * FROM {source};")
for row in cur.fetchall():
    print(row, end='\n')

insert_query = f"INSERT INTO {source} (ID, Time, {columns}, Amount, Class) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
# insert_query = f"INSERT INTO {source} (ID, Time, {columns}, Amount) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

In [77]:
def toSql(dataToSend):
    try:
        cur.execute(insert_query, dataToSend)
        conn.commit()
    except psycopg2.Error as e:
        print("Error inserting data:", e)

In [78]:
data=[]

for i in range(0, len(file_data)):
    temp=file_data.iloc[i]
    t=[]
    t.append(str(uuid.uuid4()))
    for j in range (0, len(temp)):
        t.append(temp[j])
    toSql(t)
print('done')
# 6m 15.5s :)
# 1m 6.9s :))

done


In [9]:
# cur.execute(f"SELECT * FROM {source};")
# for row in cur.fetchall():
#     print(row, end='\n')

In [10]:
# cur.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{source}'")
# print(cur.fetchall())

In [79]:
# cur.execute(f"SELECT Time FROM {source}")
# for row in cur.fetchall():
#     if(row[0]>140000):
#         print(row[0], end='\n')

conn.commit()
cur.close()
conn.close()